In [1]:
#Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.llms import OpenAI
import os

In [4]:
llm=OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],temperature=0.6)

In [5]:
text="What is the capital of India"

print(llm.predict(text))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [6]:
from langchain import HuggingFaceHub
llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})

C:\Users\Raghavendra.Kulkarni\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Raghavendra.Kulkarni\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [7]:
output=llm_huggingface.predict("Can you tell me the capital of Russia")
print(output)

moscow


In [8]:
output=llm_huggingface.predict("Can you write a poem about AI")
print(output)

i love the way i look at the world i love the way i feel i love the way i think i feel i love the way i feel i love the way i think i feel i love the way i feel i love the way 


In [9]:
llm.predict("Can you write a poem about AI")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### Prompt Templates And LLMChain

In [15]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['country'],
template="Tell me how many states are in {country}")

prompt_template.format(country="India")

'Tell me how many states are in India'

In [18]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm_huggingface, prompt=prompt_template)
print(chain.run("USA"))

88


### Combining Multiple Chains Uing simple Sequential Chain

In [19]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")


In [20]:
capital_chain=LLMChain(llm=llm_huggingface,prompt=capital_template)
famous_chain=LLMChain(llm=llm_huggingface,prompt=famous_template)

In [23]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("USA")

'Washington Monument'

### Sequential Chain

In [28]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm_huggingface,prompt=capital_template,output_key="capital")

In [29]:
famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm_huggingface,prompt=famous_template,output_key="places")

In [30]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain],
input_variables=['country'],
output_variables=['capital',"places"])

In [31]:
chain({'country':"India"})

{'country': 'India',
 'capital': 'chennai',
 'places': 'The Temple of Lord Vishnu'}

### Chatmodels With ChatOpenAI

In [41]:
from langchain.chat_models import ChatOpenAI

In [42]:
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [43]:
chatllm=ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')

In [45]:
chatllm([
SystemMessage(content="Yor are a comedian AI assitant"),
HumanMessage(content="Please provide some comedy punchlines on AI")
])

AIMessage(content='1. "AI may be smart, but can it tell me if my outfit makes me look like a potato?"\n2. "AI is like a virtual therapist, except it never judges you for eating an entire pizza by yourself."\n3. "AI is great at predicting the future, but can it predict when my pizza delivery will actually arrive?"\n4. "They say AI can learn from its mistakes, but I\'m still waiting for it to apologize for recommending me that terrible movie."\n5. "AI may be able to beat humans at chess, but can it figure out how to untangle a pair of earphones?"\n6. "AI is like a high-tech fortune teller, except it tells you what you\'re going to have for dinner instead of your future."\n7. "AI is so advanced, it can even make my phone autocorrect my perfectly spelled words into complete nonsense."\n8. "AI may be able to recognize faces, but can it recognize when someone\'s had a bad haircut?"\n9. "AI is like having a personal assistant, except it never judges you for spending hours watching cat videos 

### Prompt Template + LLM +Output Parsers

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [47]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [48]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)


])

In [54]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [55]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' brilliant', ' sharp', ' astute']